# Workflow for a multi-regional energy system

In this application of the FINE framework, a multi-regional energy system is modeled and optimized.

All classes which are available to the user are utilized and examples of the selection of different parameters within these classes are given.

The workflow is structures as follows:
1. Required packages are imported and the input data path is set
2. An energy system model instance is created
3. Commodity sources are added to the energy system model
4. Commodity conversion components are added to the energy system model
5. Commodity storages are added to the energy system model
6. Commodity transmission components are added to the energy system model
7. Commodity sinks are added to the energy system model
8. The energy system model is optimized
9. Selected optimization results are presented


# 1. Import required packages and set input data path

The FINE framework is imported which provides the required classes and functions for modeling the energy system.

In [1]:
import FINE as fn
from getData import getData
import os
cwd = os.getcwd()
data = getData()

%matplotlib inline  
%load_ext autoreload
%autoreload 2

# 2. Create an energy system model instance 

The structure of the energy system model is given by the considered locations, commodities, the number of time steps as well as the hours per time step.

The commodities are specified by a unit (i.e. 'GW_electric', 'GW_H2lowerHeatingValue', 'Mio. t CO2/h') which can be given as an energy or mass unit per hour. Furthermore, the cost unit and length unit are specified.

In [2]:
locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
commodityUnitDict = {'electricity': r'GW$_{el}$', 'methane': r'GW$_{CH_{4},LHV}$', 'biogas': r'GW$_{biogas,LHV}$',
                     'CO2': r'Mio. t$_{CO_2}$/h', 'hydrogen': r'GW$_{H_{2},LHV}$'}
commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [3]:
esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                           commodityUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='1e9 Euro', lengthUnit='km', verboseLogLevel=0)

In [4]:
CO2_reductionTarget = 1

# 3. Add commodity sources to the energy system model

## 3.1. Electricity sources

### Wind onshore

In [5]:
esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (onshore), operationRateMax'],
                  capacityMax=data['Wind (onshore), capacityMax'],
                  investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                  economicLifetime=20))

Full load hours:

In [6]:
data['Wind (onshore), operationRateMax'].sum()

cluster_0    1572.003960
cluster_1    2350.292663
cluster_2    2374.507270
cluster_3    2186.572278
cluster_4    1572.650655
cluster_5    1767.840650
cluster_6    2719.564564
cluster_7    1553.045964
dtype: float64

### Wind offshore

In [7]:
esM.add(fn.Source(esM=esM, name='Wind (offshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (offshore), operationRateMax'],
                  capacityMax=data['Wind (offshore), capacityMax'],
                  investPerCapacity=2.3, opexPerCapacity=2.3*0.02, interestRate=0.08,
                  economicLifetime=20))

Full load hours:

In [8]:
data['Wind (offshore), operationRateMax'].sum()

cluster_0       0.000000
cluster_1    4435.420314
cluster_2    4301.655834
cluster_3    3902.391858
cluster_4       0.000000
cluster_5       0.000000
cluster_6    4609.508396
cluster_7       0.000000
dtype: float64

### PV

In [9]:
esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))

Full load hours:

In [10]:
data['PV, operationRateMax'].sum()

cluster_0    1113.216464
cluster_1    1053.579422
cluster_2    1058.005181
cluster_3    1079.872237
cluster_4    1140.407380
cluster_5    1051.848141
cluster_6    1069.843344
cluster_7    1085.697466
dtype: float64

### Exisisting run-of-river hydroelectricity plants

In [11]:
esM.add(fn.Source(esM=esM, name='Existing run-of-river plants', commodity='electricity',
                  hasCapacityVariable=True,
                  operationRateFix=data['Existing run-of-river plants, operationRateFix'], tsaWeight=0.01,
                  capacityFix=data['Existing run-of-river plants, capacityFix'],
                  investPerCapacity=0, opexPerCapacity=0.208))

## 3.2. Methane (natural gas and biogas)

### Natural gas

In [12]:
esM.add(fn.Source(esM=esM, name='Natural gas purchase', commodity='methane',
                  hasCapacityVariable=False, commodityCost=0.0331*1e-3))

### Biogas

In [13]:
esM.add(fn.Source(esM=esM, name='Biogas purchase', commodity='biogas',
                  operationRateMax=data['Biogas, operationRateMax'], hasCapacityVariable=False,
                  commodityCost=0.05409*1e-3))

## 3.3 CO2

### CO2

The CO2 source is required for the case in which it can be mixed with conventional natural gas.

In [14]:
esM.add(fn.Source(esM=esM, name='CO2 from enviroment', commodity='CO2',
                  hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 4. Add conversion components to the energy system model

### Combined cycle gas turbine plants

In [15]:
esM.add(fn.Conversion(esM=esM, name='CCGT plants (methane)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625},
                      hasCapacityVariable=True,
                      investPerCapacity=0.65, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### New combined cycle gas turbine plants for biogas

In [16]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (biogas)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'biogas':-1/0.635},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### New combined cycly gas turbines for hydrogen

In [17]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### Electrolyzers

In [18]:
esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

### rSOC

In [19]:
capexRSOC=1.5

esM.add(fn.Conversion(esM=esM, name='rSOEC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.6},
                      hasCapacityVariable=True, 
                      investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
                      economicLifetime=10))

esM.add(fn.Conversion(esM=esM, name='rSOFC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True, 
                      investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
                      economicLifetime=10))

# 5. Add commodity storages to the energy system model

## 5.1. Electricity storage

### Lithium ion batteries

The self discharge of a lithium ion battery is here described as 3% per month. The self discharge per hours is obtained using the equation (1-$\text{selfDischarge}_\text{hour})^{30*24\text{h}} = 1-\text{selfDischarge}_\text{month}$.

In [20]:
esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1, doPreciseTsaModeling=False,
                   investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                   economicLifetime=22))

## 5.2. Hydrogen storage

### Hydrogen filled salt caverns
The maximum capacity is here obtained by: dividing the given capacity (which is given for methane) by the lower heating value of methane and then multiplying it with the lower heating value of hydrogen.

In [21]:
esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=133,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
                   investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
                   economicLifetime=30))

## 5.3. Methane storage

### Methane filled salt caverns

In [22]:
esM.add(fn.Storage(esM=esM, name='Salt caverns (biogas)', commodity='biogas',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=443,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (methane), capacityMax'],
                   investPerCapacity=0.00004, opexPerCapacity=0.00001, interestRate=0.08,
                   economicLifetime=30))

## 5.4 Pumped hydro storage

### Pumped hydro storage

In [23]:
esM.add(fn.Storage(esM=esM, name='Pumped hydro storage', commodity='electricity',
                   chargeEfficiency=0.88, dischargeEfficiency=0.88,
                   hasCapacityVariable=True, selfDischarge=1-(1-0.00375)**(1/(30*24)),
                   chargeRate=0.16, dischargeRate=0.12, capacityFix=data['Pumped hydro storage, capacityFix'],
                   investPerCapacity=0, opexPerCapacity=0.000153))

# 6. Add commodity transmission components to the energy system model

## 6.1. Electricity transmission

### AC cables

esM.add(fn.LinearOptimalPowerFlow(esM=esM, name='AC cables', commodity='electricity',
                                  hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix'],
                                  reactances=data['AC cables, reactances']))

In [24]:
esM.add(fn.Transmission(esM=esM, name='AC cables', commodity='electricity',
                                  hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix']))

The distances of a component are set to a normalized value of 1.


### DC cables

In [25]:
esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                        distances=data['DC cables, distances'],
                        hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))

## 6.2 Methane transmission

### Methane pipeline

In [26]:
esM.add(fn.Transmission(esM=esM, name='Pipelines (biogas)', commodity='biogas', 
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                        investPerCapacity=0.000037, investIfBuilt=0.000314,
                        interestRate=0.08, economicLifetime=40))

esM.getComponent('Pipelines (methane)')._mapC['cluster_0_cluster_4'][1]

## 6.3 Hydrogen transmission

### Hydrogen pipelines

In [27]:
esM.add(fn.Transmission(esM=esM, name='Pipelines (hydrogen)', commodity='hydrogen',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                        investPerCapacity=0.000177, investIfBuilt=0.00033,
                        interestRate=0.08, economicLifetime=40))

esM.getComponent('Pipeline (hydrogen)')._mapC

# 7. Add commodity sinks to the energy system model

## 7.1. Electricity sinks

### Electricity demand

In [28]:
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

## 7.2. Hydrogen sinks

### Fuel cell electric vehicle (FCEV) demand

In [29]:
FCEV_penetration=0.5
esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))

## 7.3. CO2 sinks

### CO2 exiting the system's boundary

In [30]:
esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 8.1 Spatial Aggregation

In [32]:
from FINE.IOManagement import xarray_io as xrio

shapefileFolder = os.path.join('InputData/SpatialData/ShapeFiles/')

inputShapefile = 'clusteredRegions.shp'

esM_aggregated = esM.spatial_aggregation(numberOfRegions=3, clusterMethod="centroid-based", 
                                         shapefileFolder=shapefileFolder, inputShapefile=inputShapefile)

aggregated_regions_FilePath = os.path.join(shapefileFolder, 'aggregated_regions.shp')
aggregated_grid_FilePath = os.path.join(shapefileFolder, 'aggregated_grid.shp')

elapsed time for distance_based_clustering: 0.00 minutes


All components are now added to the model and the model can be optimized. If the computational complexity of the optimization should be reduced, the time series data of the specified components can be clustered before the optimization and the parameter timeSeriesAggregation is set to True in the optimize call.

# 8.2 Temporal Aggregation

In [33]:
esM.cluster(numberOfTypicalPeriods=7)
esM_aggregated.cluster(numberOfTypicalPeriods=7)


Clustering time series data with 7 typical periods and 24 time steps per period...
		(3.3917 sec)


Clustering time series data with 7 typical periods and 24 time steps per period...
		(1.1220 sec)



### Optimization

In [ ]:
esM.optimize(timeSeriesAggregation=True, solver="glpk")

Time series aggregation specifications:
Number of typical periods:7, number of time steps per periods:24

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(2.3365 sec)

Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.2571 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(3.1884 sec)

Declaring sets, variables and constraints for TransmissionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.7980 sec)

Declaring shared potential constraint...
		(0.0125 sec)

Declaring commodity balances...
		(1.8821 sec)

Declaring objective function...
		(0.5900 sec)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\R2D98~1.BEE\AppData\Local\Temp\tmp23b5550v.glp

  51129: obj =   1.791856745e+02 inf =   1.269e+07 (5279) 7
  51869: obj =   1.794086859e+02 inf =   1.246e+07 (5222) 6
  52630: obj =   1.789851579e+02 inf =   1.221e+07 (5173) 7
  53396: obj =   1.785667437e+02 inf =   1.197e+07 (5155) 7
  54110: obj =   1.776796105e+02 inf =   1.192e+07 (5085) 7
  54783: obj =   1.773637365e+02 inf =   1.183e+07 (5021) 6
  55487: obj =   1.764379053e+02 inf =   1.178e+07 (5001) 6
  56213: obj =   1.766998561e+02 inf =   1.169e+07 (4742) 7
  56909: obj =   1.787729653e+02 inf =   1.159e+07 (4707) 6
  57587: obj =   1.757127489e+02 inf =   1.152e+07 (4695) 7
  58284: obj =   1.749412766e+02 inf =   1.139e+07 (4662) 7
  58970: obj =   1.737807359e+02 inf =   1.128e+07 (4610) 7
  59605: obj =   1.734409077e+02 inf =   1.123e+07 (4590) 7
  60308: obj =   1.714629669e+02 inf =   1.120e+07 (4583) 6
  60970: obj =   1.714448282e+02 inf =   1.112e+07 (4552) 6
  61644: obj =   1.703201578e+02 inf =   1.106e+07 (4538) 7
  62294: obj =   1.699001504e+02 inf =  

  84210: obj =   1.210746677e+02 inf =   8.541e+06 (4349) 6
  84840: obj =   1.206841303e+02 inf =   8.526e+06 (4349) 6
  85444: obj =   1.201588030e+02 inf =   8.498e+06 (4345) 5
  86049: obj =   1.195153425e+02 inf =   8.482e+06 (4343) 6
  86684: obj =   1.195540839e+02 inf =   8.475e+06 (4341) 6
  87292: obj =   1.187200731e+02 inf =   8.453e+06 (4338) 5
  87918: obj =   1.187633679e+02 inf =   8.435e+06 (4336) 6
  88544: obj =   1.190545085e+02 inf =   8.425e+06 (4326) 6
  89192: obj =   1.184794673e+02 inf =   8.413e+06 (4324) 6
  89786: obj =   1.179888419e+02 inf =   8.400e+06 (4320) 5
  90389: obj =   1.184577812e+02 inf =   8.394e+06 (4315) 7
  91026: obj =   1.184271029e+02 inf =   8.386e+06 (4312) 6
  91723: obj =   1.182062820e+02 inf =   6.328e+06 (3823) 7
  92332: obj =   1.181192692e+02 inf =   6.320e+06 (3772) 5
  92942: obj =   1.184492441e+02 inf =   6.310e+06 (3720) 7
  93557: obj =   1.190700964e+02 inf =   6.300e+06 (3647) 5
  94152: obj =   1.200177843e+02 inf =  

*146573: obj =   6.009245310e+01 inf =   1.261e-04 (7792) 4
*147114: obj =   5.985403818e+01 inf =   1.427e-04 (7460) 5
*147667: obj =   5.973102937e+01 inf =   1.473e-04 (6412) 5
*148195: obj =   5.959231473e+01 inf =   1.514e-04 (7918) 5
*148743: obj =   5.950397788e+01 inf =   1.709e-04 (7176) 4
*149297: obj =   5.938869177e+01 inf =   1.711e-04 (7968) 4
*149868: obj =   5.928873777e+01 inf =   1.486e-04 (6978) 5
*150413: obj =   5.910586351e+01 inf =   1.584e-04 (6094) 5
*150964: obj =   5.899842228e+01 inf =   1.461e-04 (7135) 4
*151514: obj =   5.884095965e+01 inf =   1.440e-04 (7283) 5
*152065: obj =   5.870988675e+01 inf =   1.480e-04 (7830) 5
*152622: obj =   5.856747919e+01 inf =   1.531e-04 (7104) 5
*153184: obj =   5.847759714e+01 inf =   1.584e-04 (9260) 5
*153756: obj =   5.834639631e+01 inf =   1.555e-04 (6841) 5
*154289: obj =   5.821082895e+01 inf =   1.532e-04 (9316) 4
*154849: obj =   5.812525126e+01 inf =   1.525e-04 (8441) 5
*155422: obj =   5.806958124e+01 inf =  

*208090: obj =   5.393436661e+01 inf =   2.404e-04 (6807) 6
*208652: obj =   5.388753377e+01 inf =   2.272e-04 (5977) 5
*209227: obj =   5.386003761e+01 inf =   2.290e-04 (4377) 5
*209772: obj =   5.384911187e+01 inf =   2.292e-04 (8719) 5
*210343: obj =   5.302505989e+01 inf =   2.570e-04 (6524) 5
*210796: obj =   5.300270312e+01 inf =   2.406e-04 (7271) 4
*211242: obj =   5.297624445e+01 inf =   2.139e-04 (4717) 4
*211738: obj =   5.296732108e+01 inf =   2.043e-04 (7337) 4
*212317: obj =   5.295523256e+01 inf =   2.029e-04 (6197) 5
*212882: obj =   5.293482376e+01 inf =   2.046e-04 (6761) 4
*213457: obj =   5.292539084e+01 inf =   2.052e-04 (7301) 6
*214039: obj =   5.291686102e+01 inf =   2.095e-04 (5912) 5
*214627: obj =   5.290968202e+01 inf =   2.185e-04 (5208) 5
*215224: obj =   5.290258356e+01 inf =   2.211e-04 (3513) 5
*215815: obj =   5.289489339e+01 inf =   2.136e-04 (5369) 6
*216374: obj =   5.288346004e+01 inf =   2.323e-04 (3918) 5
*216938: obj =   5.287413472e+01 inf =  

...

*3892499: obj =   6.906359848e+02 inf =   1.327e-05 (6010) 4
*3893006: obj =   6.854640225e+02 inf =   1.344e-05 (7392) 5
*3893457: obj =   6.786153448e+02 inf =   7.655e-06 (6682) 4
*3893814: obj =   6.754756057e+02 inf =   7.517e-06 (6728) 4
*3894332: obj =   6.693307522e+02 inf =   7.470e-06 (6294) 4
*3894899: obj =   6.650685297e+02 inf =   8.311e-06 (6239) 6
*3895466: obj =   6.594554946e+02 inf =   1.077e-05 (7969) 5
*3895987: obj =   6.526828426e+02 inf =   1.068e-05 (8906) 4
*3896491: obj =   7.725918582e+01 inf =   1.149e-05 (7375) 5
*3897031: obj =   7.550104338e+01 inf =   1.433e-05 (6660) 4
*3897512: obj =   7.423866344e+01 inf =   1.133e-05 (6808) 5
*3898012: obj =   7.276501120e+01 inf =   1.145e-05 (6514) 4
*3898453: obj =   7.154038435e+01 inf =   9.127e-06 (6842) 5
*3898975: obj =   7.042330466e+01 inf =   1.558e-05 (7906) 4
*3899472: obj =   6.952088437e+01 inf =   1.042e-05 (6497) 5
*3899933: obj =   6.865946719e+01 inf =   1.150e-05 (8195) 4
*3900386: obj =   6.7885

*3955785: obj =   5.363237186e+01 inf =   6.238e-05 (2227) 5
*3956372: obj =   5.362848594e+01 inf =   5.782e-05 (4902) 6
*3956881: obj =   5.362641055e+01 inf =   6.098e-05 (2079) 4
*3957387: obj =   5.362459036e+01 inf =   5.453e-05 (1877) 5
*3957956: obj =   5.362189514e+01 inf =   6.436e-05 (2438) 6
*3958566: obj =   5.362019805e+01 inf =   6.253e-05 (2104) 5
*3959187: obj =   5.361696031e+01 inf =   6.431e-05 (3304) 6
*3959813: obj =   5.361475887e+01 inf =   8.317e-05 (2130) 6
*3960483: obj =   5.361427600e+01 inf =   1.071e-04 (2077) 5
*3961134: obj =   5.361288368e+01 inf =   7.727e-05 (4123) 6
*3961751: obj =   5.361200093e+01 inf =   6.837e-05 (1941) 5
*3962370: obj =   5.361147195e+01 inf =   6.789e-05 (1375) 6
*3963021: obj =   5.361142850e+01 inf =   7.483e-05 (2) 6
 3963026: obj =   5.361142886e+01 inf =   1.730e-06 (0) 0%
+3963026: mip =   5.385335102e+01 >=   5.255682764e+01   2.4% (37; 12)
Time used: 30789.0 secs.  Memory used: 106.7 Mb.
#3964491: obj =   5.381322705e+

 4039895: obj =   8.739983783e+03 inf =   1.247e+04 (108) 4 47%
 4040607: obj =   8.737845493e+03 inf =   1.227e+04 (108) 5 34%
 4041249: obj =   8.737580497e+03 inf =   1.226e+04 (112) 5 46%
 4041895: obj =   8.716291270e+03 inf =   1.216e+04 (106) 5 43%
 4042450: obj =   8.719807959e+03 inf =   1.213e+04 (101) 4 41%
 4043021: obj =   8.716449473e+03 inf =   1.209e+04 (103) 4 48%
 4043664: obj =   8.724324787e+03 inf =   1.204e+04 (101) 5 43%
 4044393: obj =   8.727802244e+03 inf =   1.199e+04 (109) 5 33%
 4045059: obj =   8.726930998e+03 inf =   1.190e+04 (94) 5 30%
 4045681: obj =   8.727292107e+03 inf =   1.188e+04 (93) 5 48%
 4046332: obj =   8.731855011e+03 inf =   1.181e+04 (99) 5 42%
 4046976: obj =   8.733129117e+03 inf =   1.169e+04 (99) 5 50%
 4047647: obj =   8.734131020e+03 inf =   1.157e+04 (124) 5 38%
 4048303: obj =   8.732868307e+03 inf =   1.152e+04 (83) 5 45%
 4048929: obj =   8.733953302e+03 inf =   1.151e+04 (87) 4 48%
 4049654: obj =   8.732232037e+03 inf =   9.96

 4116341: obj =   4.286656031e+03 inf =   2.542e+03 (16) 5 28%
 4116898: obj =   4.290930793e+03 inf =   2.539e+03 (12) 5 31%
 4117453: obj =   4.293470096e+03 inf =   2.537e+03 (12) 4 28%
 4118032: obj =   4.269085830e+03 inf =   2.503e+03 (13) 5 31%
 4118616: obj =   4.263606867e+03 inf =   2.495e+03 (12) 5 34%
 4119183: obj =   4.263723756e+03 inf =   2.490e+03 (12) 5 41%
 4119770: obj =   4.265915138e+03 inf =   2.481e+03 (13) 4 39%
 4120354: obj =   4.273985808e+03 inf =   2.471e+03 (13) 5 32%
 4120968: obj =   4.277217577e+03 inf =   2.465e+03 (12) 5 24%
 4121579: obj =   4.279321143e+03 inf =   2.461e+03 (41) 5 18%
 4122117: obj =   4.281845469e+03 inf =   2.460e+03 (12) 5 25%
 4122683: obj =   4.284129265e+03 inf =   2.459e+03 (12) 4 29%
 4123220: obj =   4.285831455e+03 inf =   2.457e+03 (13) 5 30%
 4123804: obj =   4.006468391e+03 inf =   2.218e+03 (13) 5 19%
 4124376: obj =   4.004152311e+03 inf =   2.213e+03 (14) 5 36%
 4124927: obj =   4.004723567e+03 inf =   2.206e+03 (12

 4187647: obj =   3.003900573e+03 inf =   2.502e+02 (6) 4 31%
 4188161: obj =   3.002347490e+03 inf =   2.485e+02 (6) 4 27%
 4188645: obj =   3.000518164e+03 inf =   2.470e+02 (8) 4 29%
 4189147: obj =   2.997269909e+03 inf =   2.451e+02 (9) 5 23%
 4189454: obj =   2.995946269e+03 inf =   2.444e+02 (7) 2 27%
 4189692: obj =   2.984042382e+03 inf =   2.134e+02 (7) 2 39%
 4189950: obj =   2.983763080e+03 inf =   2.121e+02 (6) 2 46%
 4190227: obj =   2.981706330e+03 inf =   2.101e+02 (6) 3 38%
 4190518: obj =   2.980099857e+03 inf =   2.084e+02 (11) 2 43%
 4190783: obj =   2.978406871e+03 inf =   2.069e+02 (8) 2 47%
 4190994: obj =   2.976940155e+03 inf =   2.056e+02 (9) 2 38%
 4191306: obj =   2.975662857e+03 inf =   2.043e+02 (8) 3 25%
 4191695: obj =   2.974768746e+03 inf =   2.035e+02 (6) 3 31%
 4192182: obj =   2.973060761e+03 inf =   2.025e+02 (6) 4 29%
 4192649: obj =   2.971855525e+03 inf =   2.010e+02 (6) 5 37%
 4193199: obj =   2.971048179e+03 inf =   1.999e+02 (6) 4 29%
 419370

In [ ]:
esM_aggregated.optimize(timeSeriesAggregation=True, solver="glpk")

 9. Selected results output

Plot locations (GeoPandas required)

In [ ]:
locFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'clusteredRegions.shp')
fig, ax = fn.plotLocations(locFilePath, plotLocNames=True, indexColumn='index')

In [ ]:
fig, ax = fn.plotLocations(aggregated_regions_FilePath, plotLocNames=True, indexColumn='index')

### Sources and Sink

Show optimization summary

In [ ]:
esM.getOptimizationSummary("SourceSinkModel", outputLevel=2)

In [ ]:
esM_aggregated.getOptimizationSummary("SourceSinkModel", outputLevel=2)

Plot installed capacities

In [ ]:
fig, ax = fn.plotLocationalColorMap(esM, 'Wind (onshore)', locFilePath, 'index', perArea=True)

In [ ]:
fig, ax = fn.plotLocationalColorMap(esM_aggregated, 'Wind (onshore)', 
                                    aggregated_regions_FilePath, 'index', perArea=True)

In [ ]:
fig, ax = fn.plotLocationalColorMap(esM, 'PV', locFilePath, 'index', perArea=True)

In [ ]:
fig, ax = fn.plotLocationalColorMap(esM_aggregated, 'PV', aggregated_regions_FilePath, 'index', perArea=True)

Plot operation time series (either one or two dimensional)

In [ ]:
fig, ax = fn.plotOperation(esM, 'Electricity demand', 'cluster_0')

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Electricity demand', 'cluster_0')

### Conversion

Show optimization summary

In [ ]:
esM.getOptimizationSummary("ConversionModel", outputLevel=2)

In [ ]:
esM_aggregated.getOptimizationSummary("ConversionModel", outputLevel=2)

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'New CCGT plants (biogas)', 'cluster_2')

In [ ]:
fig, ax = fn.plotOperationColorMap(esM_aggregated, 'New CCGT plants (biogas)', 'cluster_1_cluster_2')

### Storage

Show optimization summary

In [ ]:
esM.getOptimizationSummary("StorageModel", outputLevel=2)

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Li-ion batteries', 'cluster_2', 
                                   variableName='stateOfChargeOperationVariablesOptimum')

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Pumped hydro storage', 'cluster_2',
                                  variableName='stateOfChargeOperationVariablesOptimum')

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Salt caverns (biogas)', 'cluster_2',
                                  variableName='stateOfChargeOperationVariablesOptimum')

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Salt caverns (hydrogen)', 'cluster_2',
                                  variableName='stateOfChargeOperationVariablesOptimum')

## Transmission

Show optimization summary

In [ ]:
esM.getOptimizationSummary("TransmissionModel", outputLevel=2).loc['Pipelines (hydrogen)']

Check that the shared capacity of the pipelines are not exceeded

In [ ]:
df=esM.componentModelingDict["TransmissionModel"].capacityVariablesOptimum
df.loc['Pipelines (biogas)']+df.loc['Pipelines (hydrogen)']

Plot installed transmission capacities

In [ ]:
transFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'AClines.shp')

fig, ax = fn.plotLocations(locFilePath, indexColumn='index')                                 
fig, ax = fn.plotTransmission(esM, 'AC cables', transFilePath, loc0='bus0', loc1='bus1', fig=fig, ax=ax)

In [ ]:
fig, ax = fn.plotLocations(aggregated_regions_FilePath, indexColumn='index')                                 
fig, ax = fn.plotTransmission(esM_aggregated, 'AC cables', aggregated_grid_FilePath, loc0='bus0', loc1='bus1', fig=fig, ax=ax)

In [ ]:
transFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'DClines.shp')

fig, ax = fn.plotLocations(locFilePath, indexColumn='index')                                 
fig, ax = fn.plotTransmission(esM, 'DC cables', transFilePath, loc0='cluster0', loc1='cluster1', fig=fig, ax=ax)

In [ ]:
transFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'transmissionPipeline.shp')

fig, ax = fn.plotLocations(locFilePath, indexColumn='index')                                 
fig, ax = fn.plotTransmission(esM, 'Pipelines (hydrogen)', transFilePath, loc0='cluster0', loc1='cluster1',
                              fig=fig, ax=ax)

In [ ]:
transFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'transmissionPipeline.shp')

fig, ax = fn.plotLocations(locFilePath, indexColumn='index')                                 
fig, ax = fn.plotTransmission(esM, 'Pipelines (biogas)', transFilePath, loc0='cluster0', loc1='cluster1',
                              fig=fig, ax=ax)